# SNIa Host Association

Contact author: Melissa Graham

Date last verified to run: Thu Dec 29, 2022

RSP environment version: Weekly 2022_40


## 1. Introduction


(1) Choose a SNIa to explore. Use truth table.

(2) Show the SNIa environ (deep coadd).

(3) Explain how there will be 'nearby' galaxy Object IDs in the future. Cite the DMTN.

(4) Get Object table data for objects in environ.

(5) Identify three nearest stars and galaxies (radial), and three nearest extended objects (separation distance).

(6) How often is the lowest separation distance galaxy the true host for SNeIa?

### 1.1. Import packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import gc

from astropy.wcs import WCS
from astropy.units import UnitsWarning
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
import astropy.units as u

from lsst.rsp import get_tap_service
import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler
import lsst.geom as geom

### 1.2. Set global parameters and functions

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')
skymap = butler.get('skyMap')

In [ ]:
service = get_tap_service()

In [ ]:
pd.set_option('display.max_rows', 20)
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u' : '#56b4e9', 'g' : '#008060', 'r' : '#ff4000',
                      'i' : '#850000', 'z' : '#6600cc', 'y' : '#000000'}
plot_filter_symbols = {'u' : 'o', 'g' : '^', 'r' : 'v', 
                       'i' : 's', 'z' : '*', 'y' : 'p'}

In [ ]:
def remove_figure(fig):
    """
    Remove a figure to reduce memory footprint.

    Parameters
    ----------
    fig: matplotlib.figure.Figure
        Figure to be removed.

    Returns
    -------
    None
    """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()       # clear the figure
    plt.close(fig)  # close the figure
    gc.collect()    # call the garbage collector

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              dataId=coaddId)

    return cutout_image

## 2. Find a low-z SNIa with a large offset from a big bright host

Start with a random bunch of true SNIa with z<0.2 near the center of the DC2 simulation area.

In [ ]:
%%time

query = "SELECT id_truth_type, id, ra, dec, truth_type, redshift, host_galaxy "\
        "FROM dp02_dc2_catalogs.TruthSummary "\
        "WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', 57.5, -36.5, 2)) = 1 "\
        "AND truth_type = 3 AND redshift < 0.2"
results = service.search(query)
del query

In [ ]:
TrueSNIa = results.to_table().to_pandas()
del results
print('len(TrueSNIa) = ', len(TrueSNIa))

In [ ]:
# TrueSNIa

Make a tuple-formatted string of the `host_galaxy` column, which is the `id` column in both the `TruthSummary` and `MatchesTruth` tables.

In [ ]:
tuple_string_hostId = '('
for i in range(len(TrueSNIa)):
    tuple_string_hostId += str(TrueSNIa.loc[i, 'host_galaxy'])
    if i < len(TrueSNIa)-1:
        tuple_string_hostId += ', '
    else:
        tuple_string_hostId += ')'

In [ ]:
# print(tuple_string_hostId)

Use the `MatchesTruth` table to retrieve the `objectId` for the hosts.

In [ ]:
%%time

query = "SELECT id, truth_type, id_truth_type, match_objectId "\
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "WHERE id IN "+tuple_string_hostId
results = service.search(query)
del query

In [ ]:
TrueSNIaHosts = results.to_table().to_pandas()
del results
print('len(TrueSNIaHosts) = ', len(TrueSNIaHosts))

In [ ]:
# TrueSNIaHosts

Make a tuple-formatted string of the `match_objectId` column. 

Notice that one of the hosts has a `match_objectId` = `<NA>`, and skip that one.

In [ ]:
tuple_string_objectId = '('
for i in range(len(TrueSNIaHosts)):
    if str(TrueSNIaHosts.loc[i, 'match_objectId']) != '<NA>':
        tuple_string_objectId += str(TrueSNIaHosts.loc[i, 'match_objectId'])
        if i < len(TrueSNIaHosts)-1:
            tuple_string_objectId += ', '
        else:
            tuple_string_objectId += ')'

In [ ]:
# print(tuple_string_objectId)

Retrieve measurements for the host galaxies from the `Object` table.

In [ ]:
%%time

query = "SELECT objectId, coord_ra, coord_dec, footprintArea, "\
        "scisql_nanojanskyToAbMag(r_cModelFlux) as r_cModelMag "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE objectId IN "+tuple_string_objectId
results = service.search(query)
del query

In [ ]:
Obj = results.to_table().to_pandas()
del results
print('len(Obj) = ', len(Obj))

In [ ]:
# Obj

Add the host information to the `TrueSNIa` table.

In [ ]:
TrueSNIa['host_objectId'] = np.zeros(len(TrueSNIa), dtype='int')
TrueSNIa['host_ra'] = np.zeros(len(TrueSNIa), dtype='float')
TrueSNIa['host_dec'] = np.zeros(len(TrueSNIa), dtype='float')
TrueSNIa['host_footprintArea'] = np.zeros(len(TrueSNIa), dtype='float')
TrueSNIa['host_r_cModelMag'] = np.zeros(len(TrueSNIa), dtype='float')
TrueSNIa['host_offset'] = np.zeros(len(TrueSNIa), dtype='float')

In [ ]:
for i in range(len(TrueSNIa)):
    str_id = str(TrueSNIa.loc[i, 'host_galaxy'])
    tx = np.where(str_id == TrueSNIaHosts.loc[:, 'id'])[0]
    if len(tx) == 1:
        tx2 = np.where(TrueSNIaHosts.loc[tx[0], 'match_objectId'] == Obj.loc[:, 'objectId'])[0]
        if len(tx2) == 1:
            TrueSNIa.loc[i, 'host_objectId'] = Obj.loc[tx2[0], 'objectId']
            TrueSNIa.loc[i, 'host_ra'] = Obj.loc[tx2[0], 'coord_ra']
            TrueSNIa.loc[i, 'host_dec'] = Obj.loc[tx2[0], 'coord_dec']
            TrueSNIa.loc[i, 'host_footprintArea'] = Obj.loc[tx2[0], 'footprintArea']
            TrueSNIa.loc[i, 'host_r_cModelMag'] = Obj.loc[tx2[0], 'r_cModelMag']
            
            coord_SNIa = SkyCoord(ra=TrueSNIa.loc[i, 'ra']*u.degree, 
                                  dec=TrueSNIa.loc[i, 'dec']*u.degree)
            coord_host = SkyCoord(ra=Obj.loc[tx2[0], 'coord_ra']*u.degree, 
                                  dec=Obj.loc[tx2[0], 'coord_dec']*u.degree)
            offset = coord_SNIa.separation(coord_host)
            TrueSNIa.loc[i, 'host_offset'] = offset.arcsec
            del coord_SNIa, coord_host, offset
        del tx2
    del tx

In [ ]:
# TrueSNIa

Plot the host galaxy brightness vs. footprint area.

In [ ]:
fig = plt.figure(figsize=(4, 2))
tx = np.where((TrueSNIa.loc[:, 'host_footprintArea'] > 1.0) 
              & (TrueSNIa.loc[:, 'host_r_cModelMag'] > 1.0))[0]
plt.plot(np.log10(TrueSNIa.loc[tx, 'host_footprintArea']), 
         TrueSNIa.loc[tx, 'host_r_cModelMag'], 'o', alpha=0.3)
plt.xlabel('log10(host_footprintArea)')
plt.ylabel('host_r_cModelMag')
plt.ylim([22,14])
plt.show()
del tx

Plot the host galaxy redshift vs. SNIa offset.

In [ ]:
fig = plt.figure(figsize=(4, 2))
tx = np.where(TrueSNIa.loc[:, 'host_offset'] > 0.001)[0]
plt.plot(TrueSNIa.loc[tx, 'redshift'], 
         np.log10(TrueSNIa.loc[tx, 'host_offset']), 'o', alpha=0.3)
plt.xlabel('redshift')
plt.ylabel('log10(host_offset)')
plt.show()
del tx

Select SNIa that are well-offset from their large bright (but not too bright) host.

In [ ]:
tx = np.where((TrueSNIa.loc[:, 'host_footprintArea'] > 25000)
              & (TrueSNIa.loc[:, 'host_r_cModelMag'] < 18)
              & (TrueSNIa.loc[:, 'host_r_cModelMag'] > 17)
              & (TrueSNIa.loc[:, 'host_offset'] > 10.0))[0]

In [ ]:
for x in tx:
    str_snra = str(np.round(float(TrueSNIa.loc[x, 'ra']), 6))
    str_sndec = str(np.round(float(TrueSNIa.loc[x, 'dec']), 6))
    str_rdeg = str(np.round(float(TrueSNIa.loc[x, 'host_offset'] / 3600.0), 6))
    query = "SELECT objectId "\
            "FROM dp02_dc2_catalogs.Object "\
            "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
            "CIRCLE('ICRS', "+str_snra+", "+str_sndec+", "+str_rdeg+")) = 1 "\
            "AND detect_isPrimary = 1 AND refExtendedness = 1"
    results = service.search(query)
    del query
    print('%3i %13s %8.6f %7.1f %5.2f %5.2f %3i' % 
          (x, TrueSNIa.loc[x, 'id_truth_type'], TrueSNIa.loc[x, 'redshift'], 
           TrueSNIa.loc[x, 'host_footprintArea'], TrueSNIa.loc[x, 'host_r_cModelMag'],
           TrueSNIa.loc[x, 'host_offset'], len(results)))
    del results, str_snra, str_sndec, str_rdeg

Let's choose to explore SNIa MS_9684_23_3. There are 16 other extended (non point source) objects between it and its host.

In [ ]:
sn_id_truth_type = 'MS_9940_41_3'

use_x = np.where(TrueSNIa.loc[:, 'id_truth_type'] == sn_id_truth_type)[0]

sn_redshift = float(TrueSNIa.loc[use_x, 'redshift'])
sn_ra = float(TrueSNIa.loc[use_x, 'ra'])
sn_dec = float(TrueSNIa.loc[use_x, 'dec'])
sn_coords = SkyCoord(ra=sn_ra*u.degree, 
                     dec=sn_dec*u.degree)
sn_host_objectId = int(TrueSNIa.loc[use_x, 'host_objectId'])
sn_hostoff = float(TrueSNIa.loc[use_x, 'host_offset'])

Forget everything we know about this SNIa's host, and try to do the association.

In [ ]:
# del TrueSNIa, TrueSNIaHosts, Obj

## 3. Explore the chosen SNIa and its environment

Retrieve all `Objects` within 1 arcminute.

In [ ]:
str_snra = str(np.round(sn_ra, 6))
str_sndec = str(np.round(sn_dec, 6))
str_rdeg = str(np.round(60.0 / 3600.0, 6))
query = "SELECT objectId, coord_ra, coord_dec, x, y, refExtendedness, "\
        "shape_xx, shape_xy, shape_yy, r_kronRad, i_kronRad "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
        "CIRCLE('ICRS', "+str_snra+", "+str_sndec+", "+str_rdeg+")) = 1 "\
        "AND detect_isPrimary = 1"
results = service.search(query)
Obj = results.to_table().to_pandas()
del results, str_snra, str_sndec, str_rdeg, query

In [ ]:
# Obj

Calculate the on-sky 2d offset between each `Object` and the SNIa.

In [ ]:
Obj['sn_offset'] = np.zeros(len(Obj), dtype='float')
for i in range(len(Obj)):
    coord_obj = SkyCoord(ra=Obj.loc[i, 'coord_ra']*u.degree,
                         dec=Obj.loc[i, 'coord_dec']*u.degree)
    Obj.loc[i, 'sn_offset'] = sn_coords.separation(coord_obj).arcsec
    del coord_obj

Calculate the separation distance using the r- and i-band Kron radii.

In [ ]:
Obj['sn_sep_r'] = np.zeros(len(Obj), dtype='float')
Obj['sn_sep_i'] = np.zeros(len(Obj), dtype='float')




In [ ]:
cutout = cutout_coadd(butler, sn_ra, sn_dec, band='r', cutoutSideLength=201)

In [ ]:
sn_radec = geom.SpherePoint(sn_ra, sn_dec, geom.degrees)

In [ ]:
wcs = cutout.getWcs()

In [ ]:
sn_xy = geom.PointI(wcs.skyToPixel(sn_radec))

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, x, y "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE objectId = "+str(sn_host_objectId)
results = service.search(query)
temp2 = results.to_table().to_pandas()

In [ ]:
fig, ax = plt.subplots()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout.image)

with display.Buffering():
    display.dot('+', sn_xy.getX(), sn_xy.getY(), ctype=afwDisplay.RED)
    for i in range(len(temp)):
        use_ctype = 'white'
        if temp.loc[i, 'refExtendedness'] == 0:
            use_ctype = 'yellow'
        elif temp.loc[i, 'refExtendedness'] == 1:
            use_ctype = 'orange'
        display.dot('o', temp.loc[i, 'x'], temp.loc[i, 'y'], size=4, ctype=use_ctype)
        del use_ctype
    for j in range(len(temp2)):
        display.dot('o', temp2.loc[j, 'x'], temp2.loc[j, 'y'], size=10, ctype='green')

plt.show()
remove_figure(fig)

In [ ]:
# sn_ra = 67.4579
# sn_dec = -44.0802
# sn_spherePoint = lsst.geom.SpherePoint(sn_ra*lsst.geom.degrees,
#                                        sn_dec*lsst.geom.degrees)
# sn_tract = skymap.findTract(sn_spherePoint).tract_id
# sn_patch = skymap.findTract(sn_spherePoint).findPatch(sn_spherePoint).getSequentialIndex()
# sn_dataId_deepCoadd = {'band': 'i', 'tract': sn_tract, 'patch': sn_patch}
# sn_deepCoadd = butler.get('deepCoadd', dataId=sn_dataId_deepCoadd)

In [ ]:
# fig = plt.figure(figsize=(10, 8))
# afw_display = afwDisplay.Display(1)
# afw_display.scale('asinh', 'zscale')
# afw_display.mtv(sn_deepCoadd.image)
# plt.gca().axis('on')

In [ ]:
### NO -- USE TAP FOR CATALOGS
### meaning you must decide in advance which columns to use

# dataId = {'tract': sn_tract, 'patch': sn_patch}
# nearby_objects = butler.get('objectTable', dataId=dataId)

In [ ]:
# nearby_objects